<a href="https://colab.research.google.com/github/guanwee-loo/Notebooks/blob/master/SAML_2_0_Vulnerability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create a presentation describing only 1 of the following application vulnerabilities. 
-	Session Fixation
-	Open Redirect
-	XSRF/CSRF
-	Absolute Path Traversal
-	Privacy Violation
-	SAML 2.0

In your PPT explain how the application could be exploited, why it happens and how it can be mitigated. You can use a diagram is the makes sense. 
Remark: Power point., limited to 10 slides.
Please add use case\s, code samples, flow chart or any other graphics to stand your points.


 # SAML 2.0 Vulnerabilities
 

# Introduction

SAML, or Security Assertion Markup Language, is a common standard that lets an identity provider (IdP) communicate securely with a service provider (SP) and pass on a user's authorization.  One use case is to use SAML to provide WebSSO. This XML-based communication usually happens through the user's browser, which allows attackers to intercept and modify it. 


#SSO Flow

![alt text](https://raw.githubusercontent.com/guanwee-loo/Notebooks/master/WebSSO_Flow.PNG)

![alt text](https://raw.githubusercontent.com/guanwee-loo/Notebooks/master/WebSSO_Flow2.PNG)


#How it works
TO be or not to be, thats the question



```
<SAMLResponse>
    <Issuer>https://idp.com/</Issuer>
    <Assertion ID="_id1234">
        <Subject>
            <NameID>user@user.com</NameID> <!-- The authenicated subject -->
        </Subject>
    </Assertion>
    <Signature>
        <SignedInfo>
            <CanonicalizationMethod Algorithm="xml-c14n11"/>
            <Reference URI="#_id1234"/>
        </SignedInfo>
        <SignatureValue>
            some base64 data that represents the signature of the assertion
        </SignatureValue>
    </Signature>
</SAMLResponse>
```
***This example response omitted a lot of information not relevant to the discussion***




One vulnerability class that affects SAML-based single sign-on (SSO) systems. This vulnerability can allow an attacker with authenticated access to trick SAML systems into authenticating as a different user without knowledge of the victim user’s password.

# XML Parsing Issue

# XML Digital Signature Issue

## Code fragment to demomstrate the behavior of extracting the NameID using Python lxml library

In [60]:
# Code fragment to demomstrate the behavior of extraction
# the NameID using Python lxml library

from defusedxml.lxml import fromstring
doc = "<NameID>user@user.com<!---->.evil.com</NameID>"
print("Original NameID = user@user.com.evil.com")
print("Tampered NameID = user@user.com<!---->.evil.com")
data = fromstring(doc)
print("Parsed NameID = " + data.text) # should return ‘user@user.com.evil.com’ 
#Instead it becomes 'user@user.com' after parsing. 

Original NameID = user@user.com.evil.com
Tampered NameID = user@user.com<!---->.evil.com
Parsed NameID = user@user.com


XML Parsling

In [0]:
!pip install signxml

In [0]:
import signxml

XML Canonicalization

**Canonical XML** is a normal form of XML, intended to allow relatively simple comparison of pairs of XML documents for equivalence; for this purpose, the Canonical XML transformation removes non-meaningful differences between the documents making them logically equivalent.

#Mitigation

In [52]:
from io import BytesIO

import lxml.etree as ET
f = StringIO("<NameID>user@user.com<!---->.evil.com</NameID>")
tree = ET.parse(f)
buffer = BytesIO()
tree.write_c14n(buffer,with_comments=False)
print(buffer.getvalue().decode("utf-8"))
buffer.seek(0)
tree.write_c14n(buffer,with_comments=True)
print(buffer.getvalue().decode("utf-8"))

<NameID>user@user.com.evil.com</NameID>
<NameID>user@user.com<!---->.evil.com</NameID>


# References

1.   https://duo.com/blog/duo-finds-saml-vulnerabilities-affecting-multiple-implementations
2.   https://null-byte.wonderhowto.com/how-to/hack-saml-single-sign-with-burp-suite-0184405
3.   http://sso-attacks.org/XML_Signature_Wrapping
4.   https://www.decalage.info/fr/python/lxml-c14n




# New Section